# Credit Card Fraud prediction  
APIfied with GCP

Dataset available at https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download

Discussion with Gemini AI at https://gemini.google.com/app/da520b9418e5c083

## Model training

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

import pickle
import os

In [2]:
data = pd.read_csv(os.path.join("data", "creditcard.csv"))

In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
X = data.drop('Class', axis=1)
y = data['Class']
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)


In [128]:
halfway = int(np.floor(len(X_holdout.index)/2))
X_test=X_holdout[:halfway]
y_test=y_holdout[:halfway]

X_validation=X_holdout[halfway:]
y_validation=y_holdout[halfway:]

In [6]:
X_train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
223361,143352.0,1.955041,-0.380783,-0.315013,0.330155,-0.509374,-0.086197,-0.627978,0.035994,1.054560,...,-0.125390,0.238197,0.968305,0.053208,-0.278602,-0.044999,-0.216780,0.045168,-0.047145,9.99
165061,117173.0,-0.400975,-0.626943,1.555339,-2.017772,-0.107769,0.168310,0.017959,-0.401619,0.040378,...,-0.470372,-0.153485,0.421703,0.113442,-1.004095,-1.176695,0.361924,-0.370469,-0.144792,45.90
238186,149565.0,0.072509,0.820566,-0.561351,-0.709897,1.080399,-0.359429,0.787858,0.117276,-0.131275,...,0.012227,-0.314638,-0.872959,0.083391,0.148178,-0.431459,0.119690,0.206395,0.070288,11.99
150562,93670.0,-0.535045,1.014587,1.750679,2.769390,0.500089,1.002270,0.847902,-0.081323,0.371579,...,-0.253757,0.063525,0.443431,-0.072754,0.448192,-0.655203,-0.181038,-0.093013,-0.064931,117.44
138452,82655.0,-4.026938,1.897371,-0.429786,-0.029571,-0.855751,-0.480406,-0.435632,1.313760,0.536044,...,-0.012320,-0.480691,-0.230369,0.250717,0.066399,0.470787,0.245335,0.286904,-0.322672,25.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,75618.0,1.173488,0.100792,0.490512,0.461596,-0.296377,-0.213165,-0.165254,0.119221,-0.114199,...,-0.157534,-0.186027,-0.574283,0.161405,-0.006140,0.091444,0.109235,-0.020922,0.003967,1.98
259178,159000.0,-0.775981,0.144023,-1.142399,-1.241113,1.940358,3.912076,-0.466107,1.360620,0.400697,...,-0.295730,0.037078,-0.019575,0.241830,0.682820,-1.635109,-0.770941,0.066006,0.137056,89.23
131932,79795.0,-0.146609,0.992946,1.524591,0.485774,0.349308,-0.815198,1.076640,-0.395316,-0.491303,...,0.007155,0.052649,0.354089,-0.291198,0.402849,0.237383,-0.398467,-0.121139,-0.196195,3.94
146867,87931.0,-2.948638,2.354849,-2.521201,-3.798905,1.866302,2.727695,-0.471769,2.217537,0.580199,...,0.417396,-0.332759,-1.047514,0.143326,0.678869,0.319710,0.426309,0.496912,0.335822,1.00


In [7]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier()

In [9]:
score=roc_auc_score(y_test, model.predict(X_test))
score

0.8803996146818449

## Model Upload

In [10]:
with open(os.path.join("models", 'model.pkl'), 'wb') as f:
  pickle.dump(model, f)

In [14]:
!gsutil -m cp -r models gs://alfsn-models/models

## Model deployment

In [16]:
import google.cloud.aiplatform as aiplatform

In [37]:
!gcloud auth application-default login

In [51]:
def create_and_deploy_endpoint(project_id, region, model_dir, endpoint_name, machine_type, min_replica_count, max_replica_count):
  """  Creates a Vertex AI Endpoint and deploys a model to it.
  This function was created with the help of Gemini AI
  
  Args:
    project_id: The Google Cloud project ID.
    region: The region where the endpoint will be created.
    model_name: The name of the Vertex AI model.
    endpoint_name: The desired name for the endpoint.
    machine_type: The machine type for the endpoint instances.
    min_replica_count: The minimum number of replicas.
    max_replica_count: The maximum number of replicas.
  """
  # init AI client
  aiplatform.init(project=project_id, location=region)

  # create model from bucket
  model = aiplatform.Model.upload(
      display_name=f"imported_{endpoint_name}",
      artifact_uri=model_dir,
      serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest"
  )
  print(f"Model imported: {model.resource_name}")
  
  # creates endpoint
  endpoint = aiplatform.Endpoint.create(display_name=endpoint_name)

  # creates a deployment spec
  deployed_model = endpoint.deploy(
      model=model, # this is a google.cloud.aiplatform.Model
      machine_type=machine_type,
      min_replica_count=min_replica_count,
      max_replica_count=max_replica_count,
      traffic_percentage=100,
      sync=True
  )

  return endpoint

In [56]:
project_id = "lang-prompt"
region = "us-central1"
model_dir = "gs://alfsn-models/models"
endpoint_name = "KNN_fraud_detect"
machine_type = "n1-standard-2"
min_replica_count = 1
max_replica_count = 2

In [57]:
endpoint = create_and_deploy_endpoint(project_id, 
                                      region, 
                                      model_dir, 
                                      endpoint_name, 
                                      machine_type,
                                      min_replica_count,
                                      max_replica_count)

Creating Model
Create Model backing LRO: projects/978857499305/locations/us-central1/models/6512461247387009024/operations/8113005789771202560
Model created. Resource name: projects/978857499305/locations/us-central1/models/6512461247387009024@1
To use this Model in another session:
model = aiplatform.Model('projects/978857499305/locations/us-central1/models/6512461247387009024@1')
Model imported: projects/978857499305/locations/us-central1/models/6512461247387009024
Creating Endpoint
Create Endpoint backing LRO: projects/978857499305/locations/us-central1/endpoints/647566169271697408/operations/2877571222953000960
Endpoint created. Resource name: projects/978857499305/locations/us-central1/endpoints/647566169271697408
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/978857499305/locations/us-central1/endpoints/647566169271697408')
Deploying Model projects/978857499305/locations/us-central1/models/6512461247387009024 to Endpoint : projects/978857499305/

In [58]:
print(f"Endpoint created: {endpoint.resource_name}")

Endpoint created: projects/978857499305/locations/us-central1/endpoints/647566169271697408


## Model access

In [93]:
def predict_old(project_id: str, location: str, endpoint_id: str, instance_data: list):
    # Initialize Vertex AI client
    aiplatform.init(project=project_id, location=location)

    # Load the endpoint
    endpoint = aiplatform.Endpoint(endpoint_id)

    # Make a prediction
    prediction = endpoint.predict(instances=instance_data)

    return prediction

In [94]:
def predict(project_id: str, 
            location: str, 
            endpoint_id: str, 
            instance_data: pd.DataFrame):
    """
    The function hits a Vertex API with a pandas DataFrame
    """
    aiplatform.init(project=project_id, location=location)

    endpoint = aiplatform.Endpoint(endpoint_id)

    # Convert DataFrame to list of dictionaries
    instances = instance_data.to_dict(orient='records')

    prediction = endpoint.predict(instances=instances)

    return prediction

In [129]:
X_validation = pd.DataFrame(scaler.transform(X_validation))

In [130]:
n=2500

prediction = predict_old(project_id= project_id,
                    location=region, 
                    endpoint_id=endpoint.resource_name, 
                    instance_data=X_validation.head(n).values.tolist() # send only some requests 
                    )

In [131]:
prediction

Prediction(predictions=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [132]:
roc_auc_score(y_validation.head(n), prediction.predictions)

0.8747996794871795